In [ ]:
import pandas as pd
import numpy as np
import psutil
import os, sys
from pathlib import Path
import pyarrow.dataset as ds

In [ ]:
path0 = '/mnt/sda1/RA5/data'
path1 = '/mnt/sda1/RA5/intermediate/siyoung/Choi_Jaedo'

In [ ]:
parquet_file = os.path.join(path0, "key_work_pair.parquet")

dataset = ds.dataset(parquet_file, format="parquet")

total_rows = dataset.count_rows(filter=ds.field("COM_ADM").is_valid())
quarter = total_rows // 4

print("총 행수:", total_rows)

def load_quarter(k):
    """
    k번째(0,1,2,3) 사분면 데이터 로드
    """
    start = k * quarter
    end   = (k+1) * quarter if k < 3 else total_rows  # 마지막은 나머지 포함
    batches = dataset.to_batches(batch_size=100_000,
                                 filter=ds.field("COM_ADM").is_valid())
    rows = []
    count = 0
    for batch in batches:
        df = batch.to_pandas()
        n = len(df)
        # 원하는 구간 안쪽 데이터만 취하기
        if count + n <= start:
            count += n
            continue
        if count >= end:
            break
        s = max(0, start - count)
        e = min(n, end - count)
        rows.append(df.iloc[s:e])
        count += n
    return pd.concat(rows, ignore_index=True)


# 총 행수: 1925071172

In [ ]:
public_list = pd.DataFrame(
    {'Inst Name'   : ['건강보험심사평가원', '국민건강보험공단', '우정정보관리원', '농림축산검역본부',
                        '한국전력기술', '한국수력원자력', '한국해양과학기술원', '국민연금공단',
                        '한국식품연구원', '한국교육개발원', '한국교육과정평가원'],
    'N of employee' : [2439, 1301, 816, 630, 2879, 1100, 616, 1218, 505, 440, 401,],

    'prev_COM_ADM' : [11710620, 11440610, 11215847, 41171560, 41463570,
                        11680580, 41271525, 11710710, 41135657, 11650651, 11140520],

    'after_COM_ADM' : [51130675, 51130675, 46170620, 47150640, 47150640, 47130315,
                        26200650, 52113670, 52710330, 43750253, 43750253],

    'actual_move_month' : ['2015/12', '2015/12', '2017/07', '2015/12', '2015/08',
                            '2016/03', '2017/12', '2015/06', '2017/09', '2016/11', '2018/01']
                        }
)

1st quarter

In [ ]:
part1 = load_quarter(0)

df = part1.merge(
    public_list[['prev_COM_ADM','after_COM_ADM']],
    left_on="COM_ADM",
    right_on="prev_COM_ADM",
    how="left"
)

In [ ]:
df['BS_YR_MON'] = pd.to_datetime(df['BS_YR_MON'].str.strip(), format='%Y%m')
df = df.sort_values(["KEY","BS_YR_MON"])

In [ ]:
next_com   = df.groupby('KEY', sort=False)['COM_ADM'].shift(-1)
next_month = df.groupby('KEY', sort=False)['BS_YR_MON'].shift(-1)

In [ ]:
moved_mask = (
    df['prev_COM_ADM'].notna() &
    df['COM_ADM'].eq(df['prev_COM_ADM']) &
    next_com.eq(df['after_COM_ADM'])
)

moved_workers_1 = (
    df.loc[moved_mask, ['KEY', 'BS_YR_MON', 'prev_COM_ADM', 'after_COM_ADM']]
      .assign(move_month = next_month.loc[moved_mask].values)
      .reset_index(drop=True)
)


moved_workers_1.to_parquet(os.path.join(path1,'moved_workers_1.parquet'))

2nd quarter

In [ ]:
df = load_quarter(1)

In [ ]:
df = df.merge(
    public_list[['prev_COM_ADM','after_COM_ADM']],
    left_on="COM_ADM",
    right_on="prev_COM_ADM",
    how="left"
)

In [ ]:
df['BS_YR_MON'] = pd.to_datetime(df['BS_YR_MON'].str.strip(), format='%Y%m')
df = df.sort_values(["KEY","BS_YR_MON"])

In [ ]:
next_com   = df.groupby('KEY', sort=False)['COM_ADM'].shift(-1)
next_month = df.groupby('KEY', sort=False)['BS_YR_MON'].shift(-1)

moved_mask = (
    df['prev_COM_ADM'].notna() &
    df['COM_ADM'].eq(df['prev_COM_ADM']) &
    next_com.eq(df['after_COM_ADM'])
)

moved_workers_2 = (
    df.loc[moved_mask, ['KEY', 'BS_YR_MON', 'prev_COM_ADM', 'after_COM_ADM']]
      .assign(move_month = next_month.loc[moved_mask].values)
      .reset_index(drop=True)
)


moved_workers_2.to_parquet(os.path.join(path1,'moved_workers_2.parquet'))

3rd quarter

In [ ]:
df = load_quarter(2)

In [ ]:
df = df.merge(
    public_list[['prev_COM_ADM','after_COM_ADM']],
    left_on="COM_ADM",
    right_on="prev_COM_ADM",
    how="left"
)

In [ ]:
df['BS_YR_MON'] = pd.to_datetime(df['BS_YR_MON'].str.strip(), format='%Y%m')
df = df.sort_values(["KEY","BS_YR_MON"])

In [ ]:
next_com   = df.groupby('KEY', sort=False)['COM_ADM'].shift(-1)
next_month = df.groupby('KEY', sort=False)['BS_YR_MON'].shift(-1)

moved_mask = (
    df['prev_COM_ADM'].notna() &
    df['COM_ADM'].eq(df['prev_COM_ADM']) &
    next_com.eq(df['after_COM_ADM'])
)

moved_workers_3 = (
    df.loc[moved_mask, ['KEY', 'BS_YR_MON', 'prev_COM_ADM', 'after_COM_ADM']]
      .assign(move_month = next_month.loc[moved_mask].values)
      .reset_index(drop=True)
)


moved_workers_3.to_parquet(os.path.join(path1,'moved_workers_3.parquet'))

4th quarter

In [ ]:
df = load_quarter(3)

In [ ]:
df = df.merge(
    public_list[['prev_COM_ADM','after_COM_ADM']],
    left_on="COM_ADM",
    right_on="prev_COM_ADM",
    how="left"
)

In [ ]:
df['BS_YR_MON'] = pd.to_datetime(df['BS_YR_MON'].str.strip(), format='%Y%m')
df = df.sort_values(["KEY","BS_YR_MON"])

In [ ]:
next_com   = df.groupby('KEY', sort=False)['COM_ADM'].shift(-1)
next_month = df.groupby('KEY', sort=False)['BS_YR_MON'].shift(-1)

moved_mask = (
    df['prev_COM_ADM'].notna() &
    df['COM_ADM'].eq(df['prev_COM_ADM']) &
    next_com.eq(df['after_COM_ADM'])
)

moved_workers_4 = (
    df.loc[moved_mask, ['KEY', 'BS_YR_MON', 'prev_COM_ADM', 'after_COM_ADM']]
      .assign(move_month = next_month.loc[moved_mask].values)
      .reset_index(drop=True)
)


moved_workers_4.to_parquet(os.path.join(path1,'moved_workers_4.parquet'))

Appending data

In [ ]:
moved_workers_1 = pd.read_parquet(os.path.join(path1,'moved_workers_1.parquet'))
moved_workers_2 = pd.read_parquet(os.path.join(path1,'moved_workers_2.parquet'))
moved_workers_3 = pd.read_parquet(os.path.join(path1,'moved_workers_3.parquet'))
moved_workers_4 = pd.read_parquet(os.path.join(path1,'moved_workers_4.parquet'))

In [ ]:
moved_workers = pd.concat(
    [moved_workers_1, moved_workers_2, moved_workers_3, moved_workers_4],
    ignore_index=True
)

moved_workers = moved_workers.merge(public_list, on=['prev_COM_ADM','after_COM_ADM'], how='left')
moved_workers['actual_move_month'] = pd.to_datetime(moved_workers['actual_move_month'], format='%Y/%m')

In [ ]:
print(moved_workers.shape)
moved_workers = moved_workers[moved_workers['move_month']>= moved_workers['actual_move_month']]
print(moved_workers.shape)


# (2269, 8)
  (2267, 8)

In [ ]:
moved_workers.groupby('Inst Name').agg({'KEY':'count', 'N of employee':'mean'})



#                 KEY	  N of employee
# Inst Name
# 건강보험심사평가원	  145	      2439.0
# 국민건강보험공단	    794	      1301.0
# 국민연금공단	       11	        1218.0
# 농림축산검역본부	    357	        630.0
# 우정정보관리원	      2	        816.0
# 한국교육개발원	      85	        440.0
# 한국교육과정평가원	  136	        401.0
# 한국수력원자력	      54	      1100.0
# 한국식품연구원	      1	          505.0
# 한국전력기술	      681	          2879.0
# 한국해양과학기술원	  1	            616.0


In [ ]:
moved_workers['diff_from_move'] = ((moved_workers['move_month'].dt.year - moved_workers['actual_move_month'].dt.year) * 12 +
                                    (moved_workers['move_month'].dt.month - moved_workers['actual_move_month'].dt.month))

print(moved_workers['diff_from_move'].describe())


# count    2267.000000
# mean       30.648434
# std        26.168734
# min         3.000000
# 25%        12.000000
# 50%        19.000000
# 75%        46.000000
# max       106.000000

In [ ]:
(moved_workers['diff_from_move'] > 60).sum()

# np.int64(383)